In [243]:
import pandas as pd
import numpy as np
#os.chdir('/home/joyvan/Project_1030')
census = pd.read_csv('./data/merge.csv')
cp_2017 = pd.read_csv('./data/NNDSS_-_Table_II._Tetanus_to_Varicella.csv')
cp_2014 = pd.read_csv('./data/NNDSS_-_Table_II._Varicella_to_West_Nile_virus_disease.csv')
vaccrates = pd.read_csv('./data/state_vacc_cov.csv')
#vaccrates = pd.read_excel(r'dataView2121_17-3.xls')#, sheet_name = "NIS-Child Trend-Varicella 2017")
#goes from 1996 to 2017, so it would be good to keep 2013, 2014, 2015, 2016, 2017
#row labels (leftmost column) are locations, percentages are listed under 

In [118]:
#census cleaning
#preserve a sample fact column
census_info = census["Fact"]

#remove repeated fact columns
cols = [c for c in census.columns if c.lower()[:4] != "fact" and c.lower()[:5]!= "value"]
census = census[cols]
#transpose and set new column names to census_info
census = census.T
census.columns = census_info
#remove extra non-data columns
census_info = census.columns[:61]
census = census[census_info]

#missing data is listed as NaN
#remove last row 
census.drop(census.tail(1).index, inplace=True)

#check for missing values
census.isnull().sum().sum()
#no missing data
del census['Total manufacturers shipments, 2012 ($1,000)']

#remove all extraneous punctuation and convert to floats
for c in census.columns:
    census[c] = census[c].str.replace('Z','0').str.replace('%','').str.replace(',','').str.replace('$','').astype(float)
census

#note that can choose cols for limited range processing by checking whether they include perc and then setting a limit
#-100 to 100
#df.replace(“”, numpy.nan, inplace=True) 

Fact,"Population estimates, July 1, 2018, (V2018)","Population estimates base, April 1, 2010, (V2018)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2018, (V2018)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","Female persons, percent","White alone, percent","Black or African American alone, percent",...,"Total nonemployer establishments, 2017","All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010"
Arkansas,3013825.0,2916028.0,3.4,2915918.0,6.3,23.3,17.0,50.9,79.1,15.7,...,205915.0,231959.0,123158.0,75962.0,35982.0,189029.0,25915.0,192988.0,56.0,52035.48
Arizona,7171646.0,6392288.0,12.2,6392017.0,6.1,22.9,17.5,50.3,82.8,5.1,...,485984.0,499926.0,245243.0,182425.0,135313.0,344981.0,46780.0,427582.0,56.3,113594.08
Alaska,737438.0,710249.0,3.8,710231.0,7.2,24.9,11.8,47.9,65.3,3.8,...,56796.0,68032.0,35402.0,22141.0,13688.0,51147.0,7953.0,56091.0,1.2,570640.95
Alabama,4887871.0,4780138.0,2.3,4779736.0,6.0,22.3,16.9,51.6,69.1,26.8,...,329993.0,374153.0,203604.0,137630.0,92219.0,272651.0,41943.0,316984.0,94.4,50645.33
Delaware,967171.0,897934.0,7.7,897934.0,5.7,21.1,18.7,51.6,69.5,23.0,...,65634.0,73418.0,38328.0,23964.0,14440.0,54782.0,7206.0,60318.0,460.8,1948.54
Connecticut,3572665.0,3574147.0,0.0,3574097.0,5.1,20.6,17.2,51.2,80.0,12.0,...,281453.0,326693.0,187845.0,106678.0,56113.0,259614.0,31056.0,281182.0,738.1,4842.36
Colorado,5695564.0,5029316.0,13.2,5029196.0,5.9,22.2,14.2,49.6,87.1,4.6,...,517350.0,547352.0,284554.0,194508.0,85849.0,442365.0,51722.0,469524.0,48.5,103641.89
California,39557045.0,37254523.0,6.2,37253956.0,6.2,22.7,14.3,50.3,72.1,6.5,...,3374050.0,3548449.0,1852580.0,1320085.0,1619857.0,1819107.0,252377.0,3176341.0,239.1,155779.22
Idaho,1754208.0,1567657.0,11.9,1567582.0,6.6,25.5,15.9,49.9,93.0,0.9,...,130699.0,146642.0,70438.0,45121.0,10592.0,130973.0,12804.0,124314.0,19.0,82643.12
Hawaii,1420491.0,1360307.0,4.4,1360301.0,6.2,21.4,18.4,49.9,25.6,2.2,...,111003.0,118454.0,60409.0,44453.0,74208.0,38510.0,11148.0,101334.0,211.8,6422.63


In [119]:
#cp_2014 cleaning
states = census.index.values
cp_2014 = cp_2014.set_index('Reporting Area') 
#cp_2014.T.head()
#remove non-varicella data
cols = [c for c in cp_2014.columns if 'varicella' in c.lower()]
#remove flag data and current week data
cols = [c for c in cols if not 'flag' in c.lower()]
cols = [c for c in cols if not 'week' in c.lower()]
#and ('week' not in c.lower()) and ('flag' not in c.lower)]
#remove duplicate rows
cp_2014 = cp_2014.drop_duplicates()
cp_2014 = cp_2014[cols]

#take maximum to get total number of cases per year
cp_2014  = cp_2014.groupby("Reporting Area").max()


#keep only states within regions
cols = [c for c in cp_2014.index.values if c.lower() in [e.lower() for e in states]]
#print(cp_2014.shape)
#cp_2014 = cp_2014.T
cp_2014 = cp_2014.T[cols]
cp_2014 = cp_2014.T
missing = list(set([s.lower() for s in states])-set([s.lower() for s in cp_2014.index.values]))
#^^^ states not in vacc. monitoring, let's append NaNs
for m in missing:
    cp_2014.append(pd.DataFrame({m:[np.NaN, np.NaN]}))
cp_2014

,"Varicella (chickenpox), Cum 2014","Varicella (chickenpox), Cum 2013"
Reporting Area,,
ALABAMA,104.0,160.0
ALASKA,70.0,61.0
ARIZONA,310.0,354.0
ARKANSAS,219.0,249.0
CALIFORNIA,29.0,30.0
COLORADO,309.0,353.0
CONNECTICUT,185.0,223.0
DELAWARE,15.0,23.0
FLORIDA,569.0,659.0


In [120]:
#cp_2017 cleaning
states = census.index.values
cp_2017 = cp_2017.set_index('Reporting Area') 
#cp_2014.T.head()
#remove non-varicella data
cols = [c for c in cp_2017.columns if 'varicella' in c.lower()]
#remove flag data and current week data
cols = [c for c in cols if not 'flag' in c.lower()]
cols = [c for c in cols if not 'week' in c.lower()]
#and ('week' not in c.lower()) and ('flag' not in c.lower)]
#remove duplicate rows
cp_2017 = cp_2017.drop_duplicates()
cp_2017 = cp_2017[cols]

#take maximum to get total number of cases per year
cp_2017  = cp_2017.groupby("Reporting Area").max()


#keep only states within regions
cols = [c for c in cp_2017.index.values if c.lower() in [e.lower() for e in states]]
#print(cp_2014.shape)
#cp_2014 = cp_2014.T
cp_2017 = cp_2017.T[cols]
cp_2017 = cp_2017.T
missing = list(set([s.lower() for s in states])-set([s.lower() for s in cp_2017.index.values])) #all states are included, though there are NaNs

cp_2017

#now let's combine the two

,"Varicella (chickenpox), Cum 2017","Varicella (chickenpox), Cum 2016"
Reporting Area,,
ALABAMA,36.0,107.0
ALASKA,47.0,19.0
ARIZONA,194.0,279.0
ARKANSAS,109.0,140.0
CALIFORNIA,24.0,44.0
COLORADO,238.0,331.0
CONNECTICUT,109.0,113.0
DELAWARE,22.0,12.0
FLORIDA,674.0,733.0


In [126]:
var_reporting = cp_2017.join(cp_2014)
var_reporting

#if states are missing their entire row, it may be prudent to exclude them from analysis. This is unfortunate,
#but it's tricky to find a good target for it.
var_reporting["Avg"] = var_reporting.mean(numeric_only=True,axis = 1)
#summary_ave_data['average'] = summary_ave_data.mean(numeric_only=True, axis=1)
var_reporting

,"Varicella (chickenpox), Cum 2017","Varicella (chickenpox), Cum 2016","Varicella (chickenpox), Cum 2014","Varicella (chickenpox), Cum 2013",Avg
Reporting Area,,,,,
ALABAMA,36.0,107.0,104.0,160.0,101.75
ALASKA,47.0,19.0,70.0,61.0,49.25
ARIZONA,194.0,279.0,310.0,354.0,284.25
ARKANSAS,109.0,140.0,219.0,249.0,179.25
CALIFORNIA,24.0,44.0,29.0,30.0,31.75
COLORADO,238.0,331.0,309.0,353.0,307.75
CONNECTICUT,109.0,113.0,185.0,223.0,157.50
DELAWARE,22.0,12.0,15.0,23.0,18.00
FLORIDA,674.0,733.0,569.0,659.0,658.75


In [152]:
#vaccrates cleaning
#remove non-state areas

#vaccrates = vaccrates.set_index("States")

cols = [c for c in vaccrates.index.values if c.lower() in [e.lower() for e in states]]
vaccratestest = vaccrates.T[cols]
rates = vaccratestest.T

rates

,n,%,CI
States,,,
Alabama,295,92,(±4)
Alaska,251,85.6,(±4.7)
Arizona,265,92.4,(±3.4)
Arkansas,258,91.2,(±4.2)
California,324,92.6,(±4.7)
Colorado,264,87.6,(±4.7)
Connecticut,266,95.4,(±3)
Delaware,232,92.1,(±3.9)
Florida,314,93.2,(±3.7)


In [165]:
#so now we have our datasets:
#rates
#census
#var_reporting
#all have 50 rows for the states.\
#so, let's join them together

#first, let's change all the state names to lowercase
census.index = census.index.str.lower()
rates.index = rates.index.str.lower()
var_reporting.index = var_reporting.index.str.lower()

df = var_reporting.join(rates)
df = df.join(census)
df

,"Varicella (chickenpox), Cum 2017","Varicella (chickenpox), Cum 2016","Varicella (chickenpox), Cum 2014","Varicella (chickenpox), Cum 2013",Avg,n,%,CI,"Population estimates, July 1, 2018, (V2018)","Population estimates base, April 1, 2010, (V2018)",...,"Total nonemployer establishments, 2017","All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010"
Reporting Area,,,,,,,,,,,,,,,,,,,,,
alabama,36.0,107.0,104.0,160.0,101.75,295,92,(±4),4887871.0,4780138.0,...,329993.0,374153.0,203604.0,137630.0,92219.0,272651.0,41943.0,316984.0,94.4,50645.33
alaska,47.0,19.0,70.0,61.0,49.25,251,85.6,(±4.7),737438.0,710249.0,...,56796.0,68032.0,35402.0,22141.0,13688.0,51147.0,7953.0,56091.0,1.2,570640.95
arizona,194.0,279.0,310.0,354.0,284.25,265,92.4,(±3.4),7171646.0,6392288.0,...,485984.0,499926.0,245243.0,182425.0,135313.0,344981.0,46780.0,427582.0,56.3,113594.08
arkansas,109.0,140.0,219.0,249.0,179.25,258,91.2,(±4.2),3013825.0,2916028.0,...,205915.0,231959.0,123158.0,75962.0,35982.0,189029.0,25915.0,192988.0,56.0,52035.48
california,24.0,44.0,29.0,30.0,31.75,324,92.6,(±4.7),39557045.0,37254523.0,...,3374050.0,3548449.0,1852580.0,1320085.0,1619857.0,1819107.0,252377.0,3176341.0,239.1,155779.22
colorado,238.0,331.0,309.0,353.0,307.75,264,87.6,(±4.7),5695564.0,5029316.0,...,517350.0,547352.0,284554.0,194508.0,85849.0,442365.0,51722.0,469524.0,48.5,103641.89
connecticut,109.0,113.0,185.0,223.0,157.50,266,95.4,(±3),3572665.0,3574147.0,...,281453.0,326693.0,187845.0,106678.0,56113.0,259614.0,31056.0,281182.0,738.1,4842.36
delaware,22.0,12.0,15.0,23.0,18.00,232,92.1,(±3.9),967171.0,897934.0,...,65634.0,73418.0,38328.0,23964.0,14440.0,54782.0,7206.0,60318.0,460.8,1948.54
florida,674.0,733.0,569.0,659.0,658.75,314,93.2,(±3.7),21299325.0,18804580.0,...,2245127.0,2100187.0,1084885.0,807817.0,926112.0,1121749.0,185756.0,1846686.0,350.6,53624.76


In [209]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#exclude all of the target variables (var_reporting.columns)
#exclude column CI
#if perec or %, minmax
#for all others, standard scaler.

Y = list(var_reporting.columns.values)
percents = [c for c in df if "perc" in c.lower() or "%" in c] 
exclude = ["CI"]
tot = Y + percents + exclude
std = list(set(df.columns) - (set(tot)))

ss = StandardScaler()
ss_vals = ss.fit_transform(df[std])

mm = MinMaxScaler()
mm_vals = mm.fit_transform(df[percents])

df_p = df.copy()
df_p[std] = ss_vals
df_p[percents] = mm_vals

#df_p

In [245]:
#OKAY COOL TIME FOR SOME EDA
import matplotlib
from matplotlib import pylab as plt

df_p["Avg"].describe()
df_p.describe()

,"Varicella (chickenpox), Cum 2017","Varicella (chickenpox), Cum 2016","Varicella (chickenpox), Cum 2014","Varicella (chickenpox), Cum 2013",Avg,n,%,"Population estimates, July 1, 2018, (V2018)","Population estimates base, April 1, 2010, (V2018)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2018, (V2018)",...,"Total nonemployer establishments, 2017","All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010"
count,36.000000,39.000000,38.000000,39.000000,39.000000,5.000000e+01,50.000000,5.000000e+01,5.000000e+01,50.000000,...,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01
mean,195.722222,229.076923,240.631579,291.000000,239.179487,3.247402e-16,0.456613,-3.108624e-17,-2.664535e-17,0.464142,...,1.554312e-17,5.107026e-17,-6.883383e-17,5.551115e-17,-2.220446e-18,3.330669e-17,-1.998401e-17,1.554312e-17,1.509903e-16,3.330669e-17
std,219.233289,261.837674,296.909798,348.305598,274.255667,1.010153e+00,0.232399,1.010153e+00,1.010153e+00,0.269716,...,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00
min,8.000000,5.000000,4.000000,5.000000,6.500000,-1.097654e+00,0.000000,-8.155240e-01,-8.259078e-01,0.000000,...,-7.195612e-01,-7.486042e-01,-7.635362e-01,-7.236439e-01,-5.229185e-01,-9.550589e-01,-8.541666e-01,-7.341942e-01,-7.496575e-01,-8.193110e-01
25%,37.500000,47.500000,47.500000,57.500000,47.500000,-5.566956e-01,0.316532,-6.430135e-01,-6.386992e-01,0.257396,...,-6.035046e-01,-6.187934e-01,-6.285433e-01,-6.118187e-01,-4.815384e-01,-6.967016e-01,-6.732469e-01,-6.078536e-01,-5.823250e-01,-3.989929e-01
50%,109.000000,149.000000,171.500000,168.000000,157.500000,-3.230998e-01,0.475806,-2.692728e-01,-2.546918e-01,0.390533,...,-3.122218e-01,-3.061156e-01,-2.924138e-01,-2.837961e-01,-3.196772e-01,-2.893824e-01,-2.415344e-01,-3.168840e-01,-3.722404e-01,-1.971157e-01
75%,279.250000,309.500000,309.750000,409.000000,343.375000,1.287236e-01,0.580645,1.254214e-01,7.629108e-02,0.653846,...,5.949473e-02,5.847800e-02,1.218372e-01,3.851136e-02,-9.505695e-02,1.885299e-01,1.503483e-01,5.104933e-02,5.624695e-02,1.246432e-01
max,953.000000,1341.000000,1557.000000,1874.000000,1431.250000,3.242309e+00,1.000000,4.525689e+00,4.585998e+00,1.000000,...,4.461215e+00,4.550557e+00,4.436985e+00,4.569354e+00,4.892107e+00,4.015541e+00,3.916048e+00,4.597095e+00,3.871042e+00,5.885642e+00


In [219]:
corr_matrix = df_p.corr()
#print(df_p.columns)
print(corr_matrix.shape)
#print(corr_matrix)
df_p.columns[0:10]
#df_p.shape


export_csv = df_p.to_csv ('Project_1030/data/df_pre.csv')#), index = None, header=True) #Don't forget to add '.csv' at the end of the path
#print (df)

(67, 67)
